In [1]:
import numpy as np
import pandas as pd

path_to_csv = "../data/mortgage_transformed.csv"
df = pd.read_csv(path_to_csv)
df.set_index(df.pop("Unnamed: 0"), inplace=True)

X = df.dropna(axis=0)
y = X.pop("result")

# More features and feature enginnering

Instead of focusing on the model right away, we will try to get most out of the data. We will fine tune the model in later notebooks.

We already have some features from the previous notebook. Let's add more

In [6]:
# features from previous notebook
X["time_since_start"] = X.time - X.first_time
X["balance_change"] = X.balance_orig_time-X.balance_time
X["LTV_change"] = X.LTV_time - X.LTV_orig_time

# new features
X["interest_rate_change"] = X.interest_rate_time - X.Interest_Rate_orig_time
X["hpi_change"] = X.hpi_time - X.hpi_orig_time

In [7]:
from sklearn.model_selection import train_test_split

desirable_columns = ["time_since_start", "balance_change", "LTV_change", "interest_rate_change", "hpi_change"]

X_train, X_valid, y_train, y_valid = train_test_split(
    X[desirable_columns], y,
    train_size=0.7,
    test_size=0.3)

In [8]:
# model scoring
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def score_model(X_train, y_train, X_valid, y_valid, n_estimators=100, learning_rate=0.1):
    model = XGBClassifier(
                n_estimators = n_estimators,
                learning_rate = learning_rate,
                n_jobs = 5,
                eval_metric='mlogloss'
            )
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    
    accuracy = accuracy_score(predictions, y_valid)
    return accuracy

C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [9]:
# we will try different number of estimators to see, which is the best fit
estimator_count = [25, 50, 100, 200, 300, 400, 800]

last_accuracy = 0
for n_estimators in estimator_count:
    accuracy = score_model(
        X_train, y_train, X_valid, y_valid,
        n_estimators=n_estimators,
        learning_rate=0.3 # let's speed things up
    )
    if accuracy > last_accuracy:
        last_accuracy = accuracy
    else:
        print(f"The ideal number of estimators is {n_estimators} with an accuracy of {accuracy}")
        break

C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and wil

The ideal number of estimators is 100 with an accuracy of 0.6607516808111981


# Results

This is not all much better than the previous one. We will try to infer hidden relationships from the data in the next notebook